In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json
from keras.models import load_model

## Train

In [2]:
def preprocess(train_path, test_path, num):
    input_texts = []
    target_texts = []
    
    test_input_texts = []
    test_target_texts = []
    
    num_lines_train_file = sum(1 for line in open(train_path, encoding='utf-8'))
    num_lines_test_file = sum(1 for line in open(test_path, encoding='utf-8'))
                    
    print("Read",train_path,"...")
    counter = 0
    with open(train_path,  encoding='utf-8') as fp:
        for json_str in fp:
            counter = counter + 1
            data = json.loads(json_str)
            input_texts.append(data["english"])
            target_texts.append(data["chinese"])
            
            '''
            if counter%1000000==0:
                print("Now processing {}/{} rows...".format(counter, num_lines_train_file))
            '''
            if counter==num:
                break
            
    print("Read",train_path,"finished!")
    
    print("\nRead",test_path,"...")
    counter = 0
    with open(test_path,  encoding='utf-8') as fp:
        for json_str in fp:
            counter = counter + 1
            data = json.loads(json_str)
            
            if counter <= num_lines_test_file-100:
                #input_texts.append(data["english"])
                #target_texts.append(data["chinese"])
                pass
            else:
                test_input_texts.append(data["english"])
                test_target_texts.append(data["chinese"])
    print("Read",test_path,"finished!")      
          
    return input_texts, target_texts, test_input_texts, test_target_texts

In [3]:
def getInputTargetChars(input_texts, target_texts):
    print("\nProcessing chars...")
    input_characters = set()
    target_characters = set()
    for input_text, target_text in zip(input_texts, target_texts):
        target_text = '\t' + target_text + '\n'
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    print("Processing chars finished!")
    return input_characters, target_characters

In [4]:
def getEncoderDecoderData(input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data, input_token_index, target_token_index):
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            #print(char)
            encoder_input_data[i, t, input_token_index[char]] = 1.
        encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
        for t, char in enumerate(target_text):
            # decoder_target_data 领先 decoder_input_data by 一个时间步。
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                # decoder_target_data 将提前一个时间步，并且将不包含开始字符。
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.
        
    return  encoder_input_data, decoder_input_data, decoder_target_data

In [5]:
def genModel(latent_dim, num_encoder_tokens, num_decoder_tokens):
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    
    encoder_states = [state_h, state_c]

    
    decoder_inputs = Input(shape=(None, num_decoder_tokens))

    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return model,encoder_model,decoder_model

In [6]:
def trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs):
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)
    
    model.save(model_path)

In [7]:
train_path = 'translation2019zh/translation2019zh_train.json'
test_path = 'translation2019zh/translation2019zh_valid.json'
model_path = 'e2c_ep5.h5'
num_line_read = 4000

batch_size = 64  
epochs = 100
latent_dim = 256 

input_texts, target_texts, test_input_texts, test_target_texts = preprocess(train_path,test_path, num_line_read)
input_characters, target_characters = getInputTargetChars(input_texts, target_texts)
input_characters = sorted(list(set(input_characters)))
target_characters = sorted(list(set(target_characters)))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('\nNumber of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

Read translation2019zh/translation2019zh_train.json ...
Read translation2019zh/translation2019zh_train.json finished!

Read translation2019zh/translation2019zh_valid.json ...
Read translation2019zh/translation2019zh_valid.json finished!

Processing chars...
Processing chars finished!

Number of samples: 4000
Number of unique input tokens: 259
Number of unique output tokens: 3374
Max sequence length for inputs: 256
Max sequence length for outputs: 142


In [8]:
encoder_input_data, decoder_input_data, decoder_target_data = getEncoderDecoderData(input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data, input_token_index, target_token_index)

In [9]:
 model,encoder_model,decoder_model = genModel(latent_dim, num_encoder_tokens, num_decoder_tokens)

In [10]:
trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs)

Epoch 1/5
50/50 [==============================] - 144s 3s/step - loss: 2.3244 - accuracy: 0.7451 - val_loss: 1.8114 - val_accuracy: 0.7612
Epoch 2/5
50/50 [==============================] - 144s 3s/step - loss: 1.7806 - accuracy: 0.7616 - val_loss: 1.8044 - val_accuracy: 0.7615
Epoch 3/5
50/50 [==============================] - 142s 3s/step - loss: 1.7403 - accuracy: 0.7621 - val_loss: 1.7523 - val_accuracy: 0.7624
Epoch 4/5
50/50 [==============================] - 146s 3s/step - loss: 1.6786 - accuracy: 0.7633 - val_loss: 1.6755 - val_accuracy: 0.7623
Epoch 5/5
50/50 [==============================] - 137s 3s/step - loss: 1.6285 - accuracy: 0.7646 - val_loss: 1.6300 - val_accuracy: 0.7635


In [13]:
trainSaveModel(model, model_path, encoder_input_data, decoder_input_data, decoder_target_data, batch_size,epochs)

Epoch 1/100
50/50 [==============================] - 147s 3s/step - loss: 2.3250 - accuracy: 0.7454 - val_loss: 1.8077 - val_accuracy: 0.7610
Epoch 2/100
50/50 [==============================] - 147s 3s/step - loss: 1.7771 - accuracy: 0.7617 - val_loss: 1.7644 - val_accuracy: 0.7615
Epoch 3/100
50/50 [==============================] - 148s 3s/step - loss: 1.7403 - accuracy: 0.7620 - val_loss: 1.7116 - val_accuracy: 0.7623
Epoch 4/100
50/50 [==============================] - 150s 3s/step - loss: 1.6758 - accuracy: 0.7631 - val_loss: 1.6478 - val_accuracy: 0.7636
Epoch 5/100
50/50 [==============================] - 149s 3s/step - loss: 1.6474 - accuracy: 0.7642 - val_loss: 1.7519 - val_accuracy: 0.7648
Epoch 6/100
50/50 [==============================] - 153s 3s/step - loss: 1.6216 - accuracy: 0.7662 - val_loss: 1.6289 - val_accuracy: 0.7657
Epoch 7/100
50/50 [==============================] - 149s 3s/step - loss: 1.5918 - accuracy: 0.7681 - val_loss: 1.6085 - val_accuracy: 0.7690
Epoch 

50/50 [==============================] - 151s 3s/step - loss: 0.7314 - accuracy: 0.8550 - val_loss: 1.4890 - val_accuracy: 0.7901
Epoch 59/100
50/50 [==============================] - 151s 3s/step - loss: 0.7194 - accuracy: 0.8568 - val_loss: 1.4939 - val_accuracy: 0.7899
Epoch 60/100
50/50 [==============================] - 153s 3s/step - loss: 0.7084 - accuracy: 0.8587 - val_loss: 1.5108 - val_accuracy: 0.7895
Epoch 61/100
50/50 [==============================] - 147s 3s/step - loss: 0.6972 - accuracy: 0.8605 - val_loss: 1.5110 - val_accuracy: 0.7893
Epoch 62/100
50/50 [==============================] - 149s 3s/step - loss: 0.6860 - accuracy: 0.8623 - val_loss: 1.5207 - val_accuracy: 0.7893
Epoch 63/100
50/50 [==============================] - 149s 3s/step - loss: 0.6748 - accuracy: 0.8645 - val_loss: 1.5214 - val_accuracy: 0.7890
Epoch 64/100
50/50 [==============================] - 153s 3s/step - loss: 0.6647 - accuracy: 0.8663 - val_loss: 1.5289 - val_accuracy: 0.7892
Epoch 65/100

## Test

In [11]:
def decode_sequence(input_seq):
    # 将输入编码为状态向量。
    states_value = encoder_model.predict(input_seq)

    # 生成长度为 1 的空目标序列。
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 用起始字符填充目标序列的第一个字符。
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 一批序列的采样循环
    # (为了简化，这里我们假设一批大小为 1)。
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 采样一个 token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 退出条件：达到最大长度或找到停止符。
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 更新目标序列（长度为 1）。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新状态
        states_value = [h, c]

    return decoded_sentence

In [12]:
# 反向查询 token 索引可将序列解码回可读的内容。
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [13]:
for seq_index in range(100):
    # 抽取一个序列（训练集的一部分）进行解码。
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Correct sentence:', target_texts[seq_index])

-
Input sentence: For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.
Decoded sentence: 们的的的的的的的的的的的的的的的的的的的的的的的的。                                                                                                                     
Correct sentence: 为了更好的锐度，但是附带的会多一些颗粒度，可以使用这个显影剂的1：1稀释液。
-
Input sentence: He calls the Green Book, his book of teachings, “the new gospel.
Decoded sentence: 们的的的的的的的的的的的的的的的的的的的的的的的的。                                                                                                                     
Correct sentence: 他还把宣扬自己思想的所谓《绿皮书》称作“新福音书”。
-
Input sentence: And the light breeze moves me to caress her long ear
Decoded sentence: 们的的的的的的的的的的的的的的的的的的的的的的的的。                                                                                                                     
Correct sentence: 微风推着我去爱抚它的长耳朵
-
Input sentence: They have the blood of martyrs is the White to flow …
Decoded sentence: 们的的的的的的

KeyboardInterrupt: 

In [13]:
target_token_index['\n']

1